<a href="https://colab.research.google.com/github/fatemafaria142/Comprehensive-Exploration-of-Large-Language-Models-in-Mental-Health-Advice-Generation/blob/main/Mental_Health_Advice_Generation_using_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


#### **Dataset Link:** https://huggingface.co/datasets/Amod/mental_health_counseling_conversations

In [2]:
from datasets import load_dataset
instruct_tune_dataset = load_dataset("Amod/mental_health_counseling_conversations")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

### **Dataset structure**
* The dataset contains two columns.

In [3]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

In [4]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Question:", data['Context'])
    print("Response:", data['Response'])
    print("\n-----------------------------\n")


Data Point 1:
Question: I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?
Response: If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our culture is so saturated with the belief that if someone doesn't feel good about themselves that this is someh

### **We will use just a small subset of the data for this training example**

In [5]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3500))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(300))

In [6]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 300
    })
})

In [7]:
def create_prompt(sample):
    """
    Update the prompt template:
    Combine both the prompt and input into a single column.
    """
    bos_token = ""
    eos_token = ""

    # Use a predefined template for instructions
    instructions_template = "Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges."



    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "### Instructions:"
    full_prompt += "\n" + instructions_template
    full_prompt += "\n\n### User:"
    full_prompt += "\n" + sample['Context']
    full_prompt += "\n\n### Counselor's Response:"
    full_prompt += "\n" + sample['Response']
    full_prompt += eos_token

    return full_prompt

In [8]:
create_prompt(instruct_tune_dataset["train"][0])

"### Instructions:\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.\n\n### User:\nI'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?\n\n### Counselor's Response:\nIf everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down a

In [9]:
create_prompt(instruct_tune_dataset["train"][1])

'### Instructions:\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you\'re all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.\n\n### User:\nI\'m going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I\'m worthless and how I shouldn\'t be here.\n   I\'ve never tried or contemplated suicide. I\'ve always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?\n\n### Counselor\'s Response:\nHello, and thank you for your question and seeking advice on this. Feelings of worthlessness is unfortunately common. In fact, most people, if not all, have felt this to some degree at some point in their life. You are not alone.\xa0Changing our feelings is like changing our thoughts - it\'s hard to do. Our mi

In [10]:
create_prompt(instruct_tune_dataset["train"][2])

"### Instructions:\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.\n\n### User:\nI'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?\n\n### Counselor's Response:\nFirst thing I'd suggest is getting the sleep you need or it will impact how you think and feel. I'd look at finding what is going well in your life and what you can be grateful for. I believe everyone has talents and wants to find their purpose in life. I think you can figure it out with some help."

In [16]:
create_prompt(instruct_tune_dataset["train"][3])

"### Instructions:\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.\n\n### User:\nI'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?\n\n### Counselor's Response:\nTherapy is essential for those that are feeling depressed and worthless. When I work with those that are experiencing concerns related to feeling of depression and issues with self esteem. I generally work with my client to help build coping skills to reduce level of depression and to assist with strengthening \xa0se

In [17]:
create_prompt(instruct_tune_dataset["train"][4])

"### Instructions:\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.\n\n### User:\nI'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?\n\n### Counselor's Response:\nI first want to let you know that you are not alone in your feelings and there is always someone there to help. You can always change your feelings and change your way of thinking by being open to trying to change. You can always make yourself available to learning new things or volunteering so that you can make a pu

In [18]:
create_prompt(instruct_tune_dataset["train"][5])

"### Instructions:\nPut yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.\n\n### User:\nI'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?\n\n### Counselor's Response:\nHeck, sure thing, hun!Feelings of 'depression' have a deeply-rooted base in physical structures that may not be functioning very well at present; and, we can certainly turn them on again using means that you are able to find around the house and with relative ease :)After that, emotional and spiritual support will be

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

In [12]:
mode_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [13]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently:**

In [15]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [19]:
prompt = """### Instruction:
Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges.### Input:
'''I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n How can I change my feeling of being worthless to everyone?'''

### Counselor's Response:"""

In [20]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


"<s> \nIt sounds like you're going through a really tough time right now. Feelings of worthlessness can be really overwhelming and difficult to deal with, but it's important to remember that everyone has their own unique qualities and strengths that make them valuable, just as they are.\n\nIt's common to have negative thoughts or feelings about ourselves, but it's important to recognize that these thoughts and feelings may not always be reflective of reality. It can be helpful to challenge these thoughts by asking yourself if there is evidence to support them or if they are based on past experiences that may not be relevant to your current situation.\n\nIt's also important to take care of yourself and engage in self-care practices that help you feel good physically and mentally. This could include things like getting enough sleep, eating a healthy diet, exercising, talking to a trusted friend or therapist, and doing things you enjoy.\n\nRemember that you don't have to go through this a

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [21]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [22]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [23]:
from transformers import TrainingArguments
output_model= "mistral_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=100,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [24]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [25]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.051700
20,1.887400
30,1.880800
40,2.012700
50,2.033100
60,1.693500
70,1.809900
80,1.645200
90,1.871200
100,1.802700


TrainOutput(global_step=100, training_loss=1.8688216590881348, metrics={'train_runtime': 1630.9531, 'train_samples_per_second': 0.245, 'train_steps_per_second': 0.061, 'total_flos': 1.74835334381568e+16, 'train_loss': 1.8688216590881348, 'epoch': 0.3})

### **Save the model**

In [26]:
trainer.save_model("mistral_instruct_generation")

In [27]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [28]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [29]:
# Example usage
prompt = '''### Instructions:\n"Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges." '''
prompt += '''\n### User:\nI've been having this ongoing problem for most of my life now. I am a young adult, and right now, driving and even being a passenger gives me terrible panic attacks and anxiety. I can't ride in the backseat or the front seat with the safety belt on. I have to lean the seat back a bit because the pressure on my back terrifies me. I miss so many opportunities to go places with friends and family because of my fears of having a panic attack. My family offers little to no support. They think it's fake or "all in my head." It makes me feel guilty and at times depressed to the point where I'd rather never wake up anymore. I am not self-harming, I just feel meaningless. This constant fear has taken over my thoughts and my life. I was such a happy person. I want my life back. I want to travel with my friends and family again, but I don't know how. I've almost given up hope on getting better and just accepting the fact that I'll live with this anxiety forever.'''
prompt += "\n\n### Counselor's Response:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> ### Instructions:
"Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges." 
### User:
I've been having this ongoing problem for most of my life now. I am a young adult, and right now, driving and even being a passenger gives me terrible panic attacks and anxiety. I can't ride in the backseat or the front seat with the safety belt on. I have to lean the seat back a bit because the pressure on my back terrifies me. I miss so many opportunities to go places with friends and family because of my fears of having a panic attack. My family offers little to no support. They think it's fake or "all in my head." It makes me feel guilty and at times depressed to the point where I'd rather never wake up anymore. I am not self-harming, I just feel meaningless. This c

### **Example No:2**

In [30]:
# Example usage
prompt = '''### Instructions:\n"Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges." '''
prompt += '''\n### User:\nI self-harm, and I stop for awhile. Then when I see something sad or depressing, I automatically want to self-harm.'''
prompt += "\n\n### Counselor's Response:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

<s> ### Instructions:
"Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges." 
### User:
I self-harm, and I stop for awhile. Then when I see something sad or depressing, I automatically want to self-harm.

### Counselor's Response:
It sounds like you are dealing with some very intense emotions, and using cutting as a way to cope with those emotions. Here are a few thoughts that may be helpful for you.

First, remember that self-harm is a serious and potentially life-threatening behavior, and it is important to take it seriously if you are cutting. Consider reaching out to your family, friends, or a healthcare professional for support. They can help you talk through your feelings, and offer guidance on how to find other ways of coping with difficult emotions

### **Example No:3**

In [31]:
# Example usage
prompt = '''### Instructions:\n"Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges." '''
prompt += '''\n### User:\nThere are many people willing to lovingly provide me with a home. I have food, clothes, and a university education, but I never feel like I belong. Even when I have a good time with people who are supposed to be close, I feel like I'm just out with friends and I never go home.'''
prompt += "\n\n### Counselor's Response:"
response = generate_response(prompt, model)

# Print the response with formatted output
print(response)

<s> ### Instructions:
"Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges." 
### User:
There are many people willing to lovingly provide me with a home. I have food, clothes, and a university education, but I never feel like I belong. Even when I have a good time with people who are supposed to be close, I feel like I'm just out with friends and I never go home.

### Counselor's Response:
It can be challenging to feel deeply connected and loved when there is such a strong desire to please others. On one hand, it can be important to be aware of others' needs and to feel grateful for all the opportunities that have come your way (you're able to support yourself and attend university, etc.). On the other hand, constantly taking care of others may be getting 